In [10]:
pip install numpy matplotlib scikit-learn


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   -------- ------------------------------- 1.6/7.8 MB 8.3 MB/s eta 0:00:01
   ------------------------ --------------- 4.7/7.8 MB 13.0 MB/s eta 0:00:01
   -------------------------- ------------- 5.2/7.8 MB 8.2 MB/s eta 0:00:01
   ----------------------------- ---------- 5.8/7.8 MB 7.2 MB/s eta 0:00:01
   --------------------------------- ------ 6.6/7.8 MB 6.7 MB/s eta 0:00:01
   ------------------------------------ --- 7.1/7.8 MB 5.9 MB/s eta 0:00:01
   ------------------------------------- -- 7.3/7.8 MB 5.2 MB/s eta 0:00:01
   ---------------------------------------- 7.8/7.8 MB 4.6

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2, DenseNet169
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os


In [ ]:
from flask import Flask, request, jsonify
from tensorflow.keras.applications import ResNet50V2, DenseNet169
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

app = Flask(__name__)

# ================================
# Define the build_model function
# ================================
def build_model(input_shape=(224, 224, 3), num_classes=2):
    inputs = Input(shape=input_shape, name="main_input")

    # Define base models
    resnet_base = ResNet50V2(weights='imagenet', include_top=False, pooling='avg', name='resnet_base')
    densenet_base = DenseNet169(weights='imagenet', include_top=False, pooling='avg', name='densenet_base')

    # Pass the input through each model independently
    resnet_out = resnet_base(inputs)
    densenet_out = densenet_base(inputs)

    # Concatenate the outputs of the models
    combined = Concatenate(name='concatenate')([resnet_out, densenet_out])

    # Add fully connected layers with dropout
    x = Dense(256, activation='relu', name='fc1')(combined)
    x = Dropout(0.5, name='dropout1')(x)
    x = Dense(128, activation='relu', name='fc2')(x)
    x = Dropout(0.5, name='dropout2')(x)

    output = Dense(num_classes, activation='softmax', name='output')(x)

    # Define and return the final model
    model = Model(inputs=inputs, outputs=output, name='RegularizedMultiBranchModel')
    return model

# ================================
# Load model weights function
# ================================
def load_model_and_predict(image_path, model_weights_path):
    # Load the model architecture
    model = build_model()
    model.load_weights(model_weights_path)
    print("Model weights loaded successfully.")

    # Preprocess the image
    def preprocess_image(image_path, target_size=(224, 224)):
        img = load_img(image_path, target_size=target_size)
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        img_array /= 255.0  # Normalize to [0,1]
        return img_array

    # Preprocess the image
    test_image = preprocess_image(image_path)

    # Make predictions
    predictions = model.predict(test_image)
    predicted_class = np.argmax(predictions, axis=1)[0]

    # Define dummy class labels for demonstration
    class_labels = {0: 'DR', 1: 'NDR'}  # Replace with your actual classes
    predicted_label = class_labels[predicted_class]
    confidence = np.max(predictions) * 100

    # Return prediction result
    return predicted_label, confidence


# ================================
# API endpoint for image prediction
# ================================
@app.route("/predict", methods=["POST"])
def predict():
    if "image" not in request.files:
        return jsonify({"error": "No image file found in the request"}), 400
    
    file = request.files["image"]

    if file.filename == '':
        return jsonify({"error": "No selected file"}), 400

    try:
        # Save the image temporarily
        image_path = "temp_image.png"
        file.save(image_path)

        # Define the model weights path
        MODEL_WEIGHTS_PATH = 'C:/Users/saadi/OneDrive/Desktop/FYP/Model_Testing/final_model_weights.weights.h5'

        # Call the prediction function
        predicted_label, confidence = load_model_and_predict(image_path, MODEL_WEIGHTS_PATH)

        # Remove the temporary image file
        os.remove(image_path)

        return jsonify({
            "prediction": predicted_label,
            "confidence": confidence
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000)


Model weights saved at C:/Users/saadi/OneDrive/Desktop/FYP/Model_Testing/final_model_weights.weights.h5
Model weights loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 20s 20s/step
Predicted Class: 0, Label: DR
Confidence: 58.64%
